In [ ]:
# Importing deep learning 
from ultralytics import YOLO
import torch 

# Other functionlaities 
import os 
import pandas as pd 
import pickle 
from sklearn.model_selection import train_test_split
import shutil

# Checking if cuda is available
print("Cuda available: ", torch.cuda.is_available())

# Reading and standartizing the labels 


In [ ]:
labels = pickle.load(open('labels.pkl', 'rb'))

# Getting all the image_name entries 
image_names = [x['image_name'] for x in labels]
image_names = list(set(image_names))
print(f"Total number of images: {len(image_names)}")

# Iterating over the image names and saving the labels for the images
# to .txt files in the labels dir 
for image_name in image_names:
    # Getting the labels for the image 
    labels_for_image = [x for x in labels if x['image_name'] == image_name]
    # Getting the label strings 
    label_polygons = [x['polygon'] for x in labels_for_image]
    # Converting from arrays to lists
    label_polygons = [x.tolist() for x in label_polygons]
    # Defining the path to labels 
    image_name_coords = image_name.split('.')[0]
    image_name_coords += '.txt'
    label_path = os.path.join('labels', image_name_coords)
    # Writing the labels to the label path 
    with open(label_path, 'w') as f:
        # Converting to strings 
        label_polygons = [str(x) for x in label_polygons]
        label_polygons = [x.replace('[', '').replace(']', '').replace(',', '') for x in label_polygons]

        # Adding the 0 label to the beginning of each label
        label_polygons = ['0 ' + x for x in label_polygons]

        # Writing to the file
        f.write('\n'.join(label_polygons))

# Creating a dataset for yolo 

In [ ]:
images = os.listdir('images')
print(f"Total number of images: {len(images)}")

# Defining the ML directory 
ml_dir = 'ml_data'

# If the ml dir already exists, we will remove it
if os.path.exists(ml_dir):
    shutil.rmtree(ml_dir)

# Defining the high level images and labels dirs 
images_dir = os.path.join(ml_dir, 'images')
labels_dir = os.path.join(ml_dir, 'labels')

# In both of the images and labels, we will create the train and val directories
# and then move the images and labels to the respective directories
images_train_dir = os.path.join(images_dir, 'train')
images_test_dir = os.path.join(images_dir, 'val')
labels_train_dir = os.path.join(labels_dir, 'train')
labels_test_dir = os.path.join(labels_dir, 'val')

# Creating the directories
os.makedirs(images_train_dir, exist_ok=True)
os.makedirs(images_test_dir, exist_ok=True)
os.makedirs(labels_train_dir, exist_ok=True)
os.makedirs(labels_test_dir, exist_ok=True)

# Splitting the images into test and train
train_images, test_images = train_test_split(images, test_size=0.2)
print(f"Number of train images: {len(train_images)}")
print(f"Number of test images: {len(test_images)}")

# Copying the images and the labels
for image in train_images:
    shutil.copy(os.path.join('images', image), images_train_dir)
    image_name_coords = image.split('.')[0]
    image_name_coords += '.txt'
    shutil.copy(os.path.join('labels', image_name_coords), labels_train_dir)

for image in test_images:
    shutil.copy(os.path.join('images', image), images_test_dir)
    image_name_coords = image.split('.')[0]
    image_name_coords += '.txt'
    shutil.copy(os.path.join('labels', image_name_coords), labels_test_dir)

# Training the YOLO model 

In [ ]:
model = YOLO('yolov8m-seg.pt')
model.train(data='segment.yaml', epochs=40, batch=4)